In [1]:

import importlib
import copy
import pickle
# --------------------
from sklearn.utils import gen_batches, check_array
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error as RMSE
from sklearn.utils.validation import FLOAT_DTYPES
import numpy as np
from numpy.linalg import norm,inv,matrix_rank, pinv
# --------------------
from skopt.space import Integer
from skopt.utils import use_named_args
from skopt import gp_minimize
# --------------------
import h5py
import datetime
import Code.SIMPLS

# import warnings
# warnings.filterwarnings('ignore')

from pytictoc import TicToc
tim=TicToc()
tim_tot = TicToc()

In [2]:
with h5py.File('./data/merra2_t.h5', 'r') as f:
    X_train, Y_train = f['X_train'], f['Y_train']
    n_train = X_train.shape[0]
    print(n_train)

    X_train = X_train[0:n_train]
    Y_train = Y_train[0:n_train]

n_fold = 74
test_size=30

4412


In [ ]:
with h5py.File('./data/TW_PM25.h5', 'r') as f:
    X_train, Y_train = f['X_train'], f['Y_train']
    n_train = X_train.shape[0]
    print(n_train)

    X_train = X_train[0:n_train]
    Y_train = Y_train[0:n_train]
n_fold = 17
test_size=30

In [3]:
def dump_SIMPLS(PLS, fn):
    np.savez(fn,
             n=PLS.n,
             n_comp=PLS.n_components,
             x_weights = PLS.x_weights_,
             x_loadings = PLS.x_loadings_,
             y_loadings = PLS.y_loadings_,
             x_mean = PLS._x_mean,
             y_mean = PLS._y_mean
             )

def predict(dat, X, n_comp):
    X = check_array(X, copy=copy, dtype=FLOAT_DTYPES)
    x_weights  = dat['x_weights'][:,:n_comp]
    x_loadings = dat['x_loadings'][:,:n_comp]
    y_loadings = dat['y_loadings'][:,:n_comp]

    x_rotaions = np.dot(
        x_weights,
        pinv(np.dot(x_loadings.T,
                    x_weights))
    )

    coef = np.dot(x_rotaions, y_loadings.T)
    X -= dat['x_mean']
    ypred = np.dot(X, coef)
    ypred += dat['y_mean']

    return ypred


In [4]:
print(X_train.shape)
print(Y_train.shape)

tscv = TimeSeriesSplit(n_splits=n_fold, test_size=test_size)
# for i, (train_index, test_index) in enumerate(tscv.split(X_train)):
#     print(f"Fold {i}:")
#     print(f"  Train: len={len(train_index)}")
#     print(f"  Test:  len={len(test_index)}")
#     if(i>3):
#         print(f"  Train: {train_index}")
#         print(f"  Test:  {test_index}")
#         break

(4412, 30456)
(4412, 30456)


In [5]:
importlib.reload(Code.SIMPLS)
from Code.SIMPLS import SIMPLS

n_comp_max=100
PLS = SIMPLS(n_components=n_comp_max)
tim_tot.tic()
tim.tic()
 for i,(train_index, test_index) in enumerate(tscv.split(X_train)):
    PLS.fit(X_train[train_index], Y_train[train_index])
    dump_SIMPLS(PLS, f'./data/wrk/SIMPLS_fold{i:02d}.npz')
    if (i%5==0):
        print(f"fold: {i+1:02d}, elapsed time: {tim.tocvalue():.1f}s")
        tim.tic()
print(f"total time={tim_tot.tocvalue():.1f}s")

fold: 01, elapsed time: 1379.8s


fold: 06, elapsed time: 8002.4s


fold: 11, elapsed time: 7251.3s


fold: 16, elapsed time: 6966.8s


fold: 21, elapsed time: 6655.1s


...

fold: 56, elapsed time: 5946.1s


fold: 61, elapsed time: 6071.8s


fold: 66, elapsed time: 5821.8s


fold: 71, elapsed time: 5937.2s


total time=95323.3s


In [6]:
PLS_list = []
for i in range(n_fold):
    PLS_list.append(np.load(f'./data/wrk/SIMPLS_fold{i:02d}.npz'))


In [9]:
space  = [Integer(1, n_comp_max, name='n_components')]
@use_named_args(space)
def Comp_Model_Score(n_components):
    scores = np.zeros((n_fold,))
    for i,(train_index, test_index) in enumerate(tscv.split(X_train)):
        y_true = Y_train[test_index]
        y_pred = predict(PLS_list[i], X_train[test_index], n_components)
        scores[i] = RMSE(y_true, y_pred)
    return np.mean(scores)


In [10]:
n_calls = int(np.log(n_comp_max)) + 1
print(f"n_calls = {n_calls:d}")
n_calls *= 2

tim_tot.tic()
res_gp = gp_minimize(Comp_Model_Score, space, n_calls=max(n_calls,10), 
                     random_state=0, verbose=True)

print("-"*40)
print(f"Best param = {res_gp.x[0]:02d}")
print(f"Best score = {res_gp.fun:.4f}")
print(f"total time = {tim_tot.tocvalue():.1f}s")


n_calls = 5
Iteration No: 1 started. Evaluating function at random point.


Iteration No: 1 ended. Evaluation done at random point.
Time taken: 228.8068
Function value obtained: 2.1518
Current minimum: 2.1518
Iteration No: 2 started. Evaluating function at random point.


Iteration No: 2 ended. Evaluation done at random point.
Time taken: 228.6431
Function value obtained: 2.1548
Current minimum: 2.1518
Iteration No: 3 started. Evaluating function at random point.


Iteration No: 3 ended. Evaluation done at random point.
Time taken: 230.6949
Function value obtained: 2.1561
Current minimum: 2.1518
Iteration No: 4 started. Evaluating function at random point.


Iteration No: 4 ended. Evaluation done at random point.
Time taken: 238.3250
Function value obtained: 2.1548
Current minimum: 2.1518
Iteration No: 5 started. Evaluating function at random point.


...

Iteration No: 6 ended. Evaluation done at random point.
Time taken: 239.9568
Function value obtained: 2.1936
Current minimum: 2.1495
Iteration No: 7 started. Evaluating function at random point.


Iteration No: 7 ended. Evaluation done at random point.
Time taken: 239.0763
Function value obtained: 2.2463
Current minimum: 2.1495
Iteration No: 8 started. Evaluating function at random point.


Iteration No: 8 ended. Evaluation done at random point.
Time taken: 238.7499
Function value obtained: 2.9291
Current minimum: 2.1495
Iteration No: 9 started. Evaluating function at random point.


Iteration No: 9 ended. Evaluation done at random point.
Time taken: 238.4164
Function value obtained: 2.2605
Current minimum: 2.1495
Iteration No: 10 started. Evaluating function at random point.


Iteration No: 10 ended. Evaluation done at random point.
Time taken: 240.4557
Function value obtained: 2.1711
Current minimum: 2.1495
----------------------------------------
Best param = 63
Best score = 2.1495
total time = 2362.6s


In [ ]:
def gen_slice_month(tim_st,tim_ed):
    tim = datetime.datetime.strptime(tim_st, "%Y%m%d")
    date_ed = datetime.datetime.strptime(  tim_ed, "%Y%m%d")
    ind_st = 0
    batches  = []
    trainset = []
    samples  = []
    count = 0
    while (tim < date_ed ):
        Year = tim.strftime("%Y")
        Month = tim.strftime("%m")
        if Month == '12':
            NextYYMM = "{:04d}01".format(int(tim.strftime("%Y"))+1)
        else:
            NextYYMM = Year+"{:02d}".format(int(tim.strftime("%m"))+1)
        tmp = datetime.datetime.strptime(  NextYYMM+"01" , "%Y%m%d")
        if (tmp > date_ed):
            tmp = date_ed
        days = (tmp - tim ).days
        ind_ed = ind_st + days
        batches.append(slice(ind_st,ind_ed))
        trainset.append(slice(ind_ed))
        samples.append(ind_ed)
        ind_st = ind_ed
        tim += datetime.timedelta(days=days)
    return batches,trainset,samples

In [ ]:
tim_st = '20110401'
tim_ed = '20230430'
batches, trainset, samples = gen_slice_month(tim_st,tim_ed)
train_ind = trainset[-1]

def RunExp(method, PLS, case, save):
    tim = TicToc()
    timlist=np.zeros((2, len(batches)))
    timlist[0] = np.array(samples)
    if (case==1):
        for i,s in enumerate(batches):
            tim.tic()
            PLS.fit(X_train[s], Y_train[s])
            timlist[1,i] = tim.tocvalue()
    else:
        for i,s in enumerate(trainset):
            tim.tic()
            PLS.fit(X_train[s], Y_train[s])
            timlist[1,i] = tim.tocvalue()

    if (method != 'NIPALS'):
        PLS._comp_coef()
    coef = PLS.coef_

    if (save==1):
        with h5py.File(f"./Results/PLS2_merra2_{method}.h5", "w") as f:
            f_coef = f.create_dataset('coef', data=coef, maxshape=coef.shape, chunks=True)
            f_tim = f.create_dataset('timer', data=timlist, maxshape=timlist.shape, chunks=True)

def Read_Results(method):
    with h5py.File(f"./Results/PLS2_merra2_{method}.h5", "r") as f:
        dat={'timer': f['timer'][:],
             'coef':  f['coef'][:]}
    return dat

In [ ]:
importlib.reload(Code.SIMPLS)
from Code.SIMPLS import SIMPLS

save=1
tim.tic()
PLS= SIMPLS(n_components=63)
RunExp('SIMPLS', PLS, 1, save)
print("elapsed time:", tim.tocvalue() )